# 1. Importações


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import time
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn import svm
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import warnings
warnings.filterwarnings("ignore")


# 2. Leitura dos Dados

In [ ]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/01. Dados para treino e teste/'
cols = pd.read_csv(path + 'x_train.csv', nrows=1).columns
x_train = pd.read_csv(path + 'x_train.csv', index_col=False, usecols=cols[:-1])
x_test = pd.read_csv(path + 'x_test.csv', index_col=False, usecols=cols[:-1])
y_train = pd.read_csv(path + 'y_train.csv', index_col=False, usecols=['classification'])
y_test = pd.read_csv(path + 'y_test.csv', index_col=False, usecols=['classification'])

len(x_train), len(x_test), len(y_train), len(y_test)
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((1106, 300), (277, 300), (1106, 1), (277, 1))

# 3. SVM

In [ ]:
# Parâmetros para o grid search
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 50, 100]},
    {'kernel': ['rbf'], 'C': [0.1, 50, 100], 'gamma': [0.0001, 0.01, 1, 10] },
    {'kernel': ['poly'], 'C': [0.1, 50, 100], 'degree':np.arange(3, 7) }
]

scoring = {
    'AUC': 'roc_auc', 
    'Accuracy': make_scorer(accuracy_score)
}


In [ ]:
grid = GridSearchCV(SVC(),param_grid, verbose=2, n_jobs=-1)

In [ ]:
%time best_model = grid.fit(x_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 10.2min finished


CPU times: user 18.9 s, sys: 57.9 ms, total: 18.9 s
Wall time: 10min 27s


In [ ]:
print(f'Best score: {best_model.best_score_}')
print(f'Best model: {best_model.best_params_}')

Best score: 0.8237006237006236
Best model: {'C': 0.1, 'kernel': 'linear'}


In [ ]:
pred_test = best_model.predict(x_test)
pred_train = best_model.predict(x_train)

print('Train Accuracy: ', accuracy_score(y_train, pred_train))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test))

print('\nConfusion Matrix:')
print(confusion_matrix(y_test,pred_test))

print('\nClassification Report:')
report = classification_report(y_test, pred_test, output_dict=True)

print(report)

Train Accuracy:  0.9385171790235082
Test Accuraccy:  0.8411552346570397

Confusion Matrix:
[[111  19]
 [ 25 122]]

Classification Report:
{'0': {'precision': 0.8161764705882353, 'recall': 0.8538461538461538, 'f1-score': 0.8345864661654135, 'support': 130}, '1': {'precision': 0.8652482269503546, 'recall': 0.8299319727891157, 'f1-score': 0.8472222222222222, 'support': 147}, 'accuracy': 0.8411552346570397, 'macro avg': {'precision': 0.8407123487692949, 'recall': 0.8418890633176348, 'f1-score': 0.8409043441938179, 'support': 277}, 'weighted avg': {'precision': 0.8422181607876271, 'recall': 0.8411552346570397, 'f1-score': 0.8412920840006153, 'support': 277}}


In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'],
    'f1score': report['weighted avg']['f1-score'],
    'y_predict': [pred_test],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(best_model.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/7. SVM/svm.json')

# 4. XGBoostClassifier
**Necessário habilitar a GPU para executar**


In [ ]:
# create a default XGBoost classifier
model = XGBClassifier(
    tree_method = "gpu_hist", 
    random_state=300, 
    eval_metric=["error", "auc"]
)

In [ ]:
# Create the grid search parameter grid and scoring funcitons
param_grid = {
    "learning_rate": [0.1, 0.01],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "reg_lambda": [1, 1.5, 2],
    "gamma": [0, 0.1, 0.3],
}
scoring = {
    'AUC': 'roc_auc', 
    'Accuracy': make_scorer(accuracy_score)
}

In [ ]:
gridXGB = GridSearchCV(model,param_grid, verbose=2, n_jobs=-1)

In [ ]:
# fit grid search
%time best_model = gridXGB.fit(x_train,y_train)

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 40.7min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 55.8min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed: 72.7min
[Parallel(n_jobs=-1)]: Done 3277 tasks      | elapsed: 93.3min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 114.5min
[Parallel(n_jobs=-1)]: Done 4897 tasks      | elapsed: 140.4min
[Parallel(n_jobs=-1)]: Done 5828 tasks      | elapsed: 166.7min
[Parallel(n_jobs=-1)]: Done 6841 tasks      | elapsed: 196.7min
[Parallel(n_jobs=-1)]: Done 7934 tasks      | elapsed: 228.4min


In [ ]:
print(f'Best score: {best_model.best_score_}')
print(f'Best model: {best_model.best_params_}')

In [ ]:
{
    'Best score': 0.8499408911173617,
    'Best model': {'colsample_bytree': 0.6, 'gamma': 0.3, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 400, 'reg_lambda': 1, 'subsample': 0.6}
}

{'Best model': {'colsample_bytree': 0.6,
  'gamma': 0.3,
  'learning_rate': 0.1,
  'max_depth': 2,
  'n_estimators': 400,
  'reg_lambda': 1,
  'subsample': 0.6},
 'Best score': 0.8499408911173617}

In [ ]:
pred_test = best_model.predict(x_test)
pred_train = best_model.predict(x_train)
print('Train Accuracy: ', accuracy_score(y_train, pred_train))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test,pred_test))
print('\nClassification Report:')
report = classification_report(y_test, pred_test, output_dict=True)
print(report)

In [ ]:
{'Train Accuracy':  0.9963833634719711,
'Test Accuraccy':  0.8664259927797834,

'Confusion Matrix':
[[115,  15], [ 22, 125]],

'Classification Report':
{'0': {'precision': 0.8394160583941606, 'recall': 0.8846153846153846, 'f1-score': 0.8614232209737828, 'support': 130}, '1': {'precision': 0.8928571428571429, 'recall': 0.8503401360544217, 'f1-score': 0.8710801393728222, 'support': 147}, 'accuracy': 0.8664259927797834, 'macro avg': {'precision': 0.8661366006256517, 'recall': 0.8674777603349031, 'f1-score': 0.8662516801733025, 'support': 277}, 'weighted avg': {'precision': 0.8677764894990645, 'recall': 0.8664259927797834, 'f1-score': 0.8665480116043199, 'support': 277}}
}

{'Classification Report': {'0': {'f1-score': 0.8614232209737828,
   'precision': 0.8394160583941606,
   'recall': 0.8846153846153846,
   'support': 130},
  '1': {'f1-score': 0.8710801393728222,
   'precision': 0.8928571428571429,
   'recall': 0.8503401360544217,
   'support': 147},
  'accuracy': 0.8664259927797834,
  'macro avg': {'f1-score': 0.8662516801733025,
   'precision': 0.8661366006256517,
   'recall': 0.8674777603349031,
   'support': 277},
  'weighted avg': {'f1-score': 0.8665480116043199,
   'precision': 0.8677764894990645,
   'recall': 0.8664259927797834,
   'support': 277}},
 'Confusion Matrix': [[115, 15], [22, 125]],
 'Test Accuraccy': 0.8664259927797834,
 'Train Accuracy': 0.9963833634719711}

# Salvando os resultados


In [ ]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'],
    'f1score': report['weighted avg']['f1-score'],
    'y_predict': [pred_test],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(best_model.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/10. XGBooster/xgbooster.json')

In [ ]:
{'0': {'precision': 0.8444444444444444, 'recall': 0.8769230769230769, 'f1-score': 0.860377358490566, 'support': 130}, '1': {'precision': 0.8873239436619719, 'recall': 0.8571428571428571, 'f1-score': 0.8719723183391004, 'support': 147}, 'accuracy': 0.8664259927797834, 'macro avg': {'precision': 0.8658841940532082, 'recall': 0.8670329670329671, 'f1-score': 0.8661748384148331, 'support': 277}, 'weighted avg': {'precision': 0.8671999909606052, 'recall': 0.8664259927797834, 'f1-score': 0.8665306404318459, 'support': 277}}

{'0': {'f1-score': 0.860377358490566,
  'precision': 0.8444444444444444,
  'recall': 0.8769230769230769,
  'support': 130},
 '1': {'f1-score': 0.8719723183391004,
  'precision': 0.8873239436619719,
  'recall': 0.8571428571428571,
  'support': 147},
 'accuracy': 0.8664259927797834,
 'macro avg': {'f1-score': 0.8661748384148331,
  'precision': 0.8658841940532082,
  'recall': 0.8670329670329671,
  'support': 277},
 'weighted avg': {'f1-score': 0.8665306404318459,
  'precision': 0.8671999909606052,
  'recall': 0.8664259927797834,
  'support': 277}}

In [ ]:
'''num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds)

# create the grid search object
n_iter=50
grid = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_grid,
    scoring=scoring,
    cv=kfold,
    n_jobs=-1,
    n_iter=n_iter,
    refit="AUC",
)
num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds)

# create the grid search object
n_iter=50
grid = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_grid,
    scoring=scoring,
    cv=kfold,
    n_jobs=-1,
    n_iter=n_iter,
    refit="AUC",
)'''

SyntaxError: ignored

In [ ]:
y_pred = [1,0,0,1,1,1,1,0,0,0,1,0,0,1,1,0,1,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,0,1,1,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,0,1,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,1,1,0,1,0,1,1,0,1,0,0,1]
y_real = [1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,0,1,1,0,1,1,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,0,1,1,1,1,1,0,0,1,0,0,0,1,1,0,1,0,1,1,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,0,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1]


print('Test Accuraccy: ', accuracy_score(y_real, y_pred))

Test Accuraccy:  0.8664259927797834
